In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import lightgbm
from sklearn.metrics import mean_squared_error
import warnings
import gc


warnings.filterwarnings('ignore')
np.random.seed(4590)

/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
sales_train = pd.read_csv('/Users/sinsakuokazaki/Project/pred-future-sales/\
input/competitive-data-science-predict-future-sales/sales_train.csv')

In [3]:
test = pd.read_csv('/Users/sinsakuokazaki/Project/pred-future-sales/\
input/competitive-data-science-predict-future-sales/test.csv')

In [4]:
train = pd.read_csv('/Users/sinsakuokazaki/Project/pred-future-sales/\
input/competitive-data-science-predict-future-sales/train_total.csv')

In [5]:
month = train.columns[2:36]
for fold, i in enumerate(range(len(month))):
    use_col = ["shop_id", "item_id", "item_name", "item_category_id", \
               "item_category_name", "shop_name", "combination_id", month[i]]
    print('fold {}'.format(fold))
    train_month = train[train[month[i]].notnull()][use_col]
    train_month = train_month.rename(columns={month[i]: "total_sales"})
    train_month["date_block_num"] = i
    if i == 0:
        matrix = train_month
        continue
    matrix = pd.concat([matrix, train_month])

fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9
fold 10
fold 11
fold 12
fold 13
fold 14
fold 15
fold 16
fold 17
fold 18
fold 19
fold 20
fold 21
fold 22
fold 23
fold 24
fold 25
fold 26
fold 27
fold 28
fold 29
fold 30
fold 31
fold 32
fold 33


In [6]:
test = test.drop(columns=["ID", "in_train"])

In [7]:
test["date_block_num"] = 34

In [8]:
matrix = pd.concat([matrix, test], join="outer")

In [9]:
item_cnt_agg = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'])['item_cnt_day'].sum()

In [10]:
item_cnt_df = pd.DataFrame(item_cnt_agg).reset_index()

In [11]:
matrix = pd.merge(matrix, item_cnt_df, how="left", on=["shop_id", "item_id", "date_block_num"])

In [12]:
group = matrix.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': 'sum'})
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)
matrix = pd.merge(matrix, group, on=['date_block_num','shop_id','item_id'], how='left')
matrix['item_cnt_month'] = matrix['item_cnt_month'].fillna(0)

In [13]:
def mean_encode(statistic, group, col_name, matrix):
    group_df = matrix.groupby(group).agg({'item_cnt_month': [statistic]})
    column = col_name.format(statistic)
    group_df.columns = [ column ]
    group_df.reset_index(inplace=True)
    return group_df, column

In [14]:
def lag_mean_encoding(lag, group_df, column, group, matrix):
    tmp = group_df.copy()
    tmp['date_block_num'] += lag
    new_name = column + str(lag)
    tmp = tmp.rename(index=str, columns={column: new_name})
    print(new_name)
    matrix = pd.merge(matrix, tmp, on=group, how='left')
    return matrix

In [15]:
groups = [["date_block_num"], ["date_block_num", "item_id"], ["date_block_num", "shop_id"],
         ["date_block_num", "item_category_id"], ["date_block_num", "item_category_id", "shop_id"]]

col_names = ['date_{}_item_cnt', 'date_item_{}_item_cnt', 'date_shop_{}_item_cnt',
             'date_category_{}_item_cnt', 'date_category_shop_{}_item_cnt']

for s in ["mean", "std", "median"]:
    for g, c in zip(groups, col_names):
        group_df, column = mean_encode(s, g, c, matrix)
        for l in [2, 3, 6, 12]:
            matrix = lag_mean_encoding(l, group_df, column, g, matrix)

date_mean_item_cnt2
date_mean_item_cnt3
date_mean_item_cnt6
date_mean_item_cnt12
date_item_mean_item_cnt2
date_item_mean_item_cnt3
date_item_mean_item_cnt6
date_item_mean_item_cnt12
date_shop_mean_item_cnt2
date_shop_mean_item_cnt3
date_shop_mean_item_cnt6
date_shop_mean_item_cnt12
date_category_mean_item_cnt2
date_category_mean_item_cnt3
date_category_mean_item_cnt6
date_category_mean_item_cnt12
date_category_shop_mean_item_cnt2
date_category_shop_mean_item_cnt3
date_category_shop_mean_item_cnt6
date_category_shop_mean_item_cnt12
date_std_item_cnt2
date_std_item_cnt3
date_std_item_cnt6
date_std_item_cnt12
date_item_std_item_cnt2
date_item_std_item_cnt3
date_item_std_item_cnt6
date_item_std_item_cnt12
date_shop_std_item_cnt2
date_shop_std_item_cnt3
date_shop_std_item_cnt6
date_shop_std_item_cnt12
date_category_std_item_cnt2
date_category_std_item_cnt3
date_category_std_item_cnt6
date_category_std_item_cnt12
date_category_shop_std_item_cnt2
date_category_shop_std_item_cnt3
date_category

In [17]:
matrix = matrix.fillna(0)

In [20]:
matrix = matrix[matrix['date_block_num'] > 11]

In [21]:
matrix.head()

,combination_id,date_block_num,item_category_id,item_category_name,item_id,item_name,shop_id,shop_name,total_sales,item_cnt_day,...,date_shop_median_item_cnt6,date_shop_median_item_cnt12,date_category_median_item_cnt2,date_category_median_item_cnt3,date_category_median_item_cnt6,date_category_median_item_cnt12,date_category_shop_median_item_cnt2,date_category_shop_median_item_cnt3,date_category_shop_median_item_cnt6,date_category_shop_median_item_cnt12
687724,"(2, 33)",12,37,Кино - Blu-Ray,33,1+1 (BD),2,"Адыгея ТЦ ""Мега""",199.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
687725,"(2, 482)",12,73,Программы - 1С:Предприятие 8,482,1С:Бухгалтерия 8. Базовая версия,2,"Адыгея ТЦ ""Мега""",6600.0,2.0,...,1.0,1.0,1.0,1.0,1.0,2.0,1.5,1.5,1.0,1.0
687726,"(2, 804)",12,49,Книги - Методические материалы 1С,804,1С:Предприятие 8.2. Практическое пособие разра...,2,"Адыгея ТЦ ""Мега""",240.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
687727,"(2, 839)",12,73,Программы - 1С:Предприятие 8,839,1С:Упрощенка 8,2,"Адыгея ТЦ ""Мега""",3300.0,1.0,...,1.0,1.0,1.0,1.0,1.0,2.0,1.5,1.5,1.0,1.0
687728,"(2, 1007)",12,67,Подарки - Развитие,1007,3D Crystal Puzzle Замок XL,2,"Адыгея ТЦ ""Мега""",599.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [36]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,dayofweek,dayofmonth,month,quarter,year,minute,hour,total_sales,shop_name,item_name,item_category_id,item_category_name
0,2013-01-02,0,59,22154,999.00,1.0,2,2,1,1,2013,0,0,999.00,"Ярославль ТЦ ""Альтаир""",ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
1,2013-01-03,0,25,2552,899.00,1.0,3,3,1,1,2013,0,0,899.00,"Москва ТРК ""Атриум""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
2,2013-01-05,0,25,2552,899.00,-1.0,5,5,1,1,2013,0,0,-899.00,"Москва ТРК ""Атриум""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
3,2013-01-06,0,25,2554,1709.05,1.0,6,6,1,1,2013,0,0,1709.05,"Москва ТРК ""Атриум""",DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил
4,2013-01-15,0,25,2555,1099.00,1.0,1,15,1,1,2013,0,0,1099.00,"Москва ТРК ""Атриум""",DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства


In [48]:
matrix = pd.merge(matrix,
                 sales_train.drop(['minute', 'hour', 'total_sales', 'shop_name', 
                                   'item_name','item_category_id','item_category_name'], axis=1), 
                 on=['date_block_num', 'shop_id', 'item_id', 'item_cnt_day'], 
                 how='left')

In [ ]:
matrix[matrix['date_block_num'] == 34]['month'] = 

In [49]:
matrix.to_pickle('/Users/sinsakuokazaki/Project/pred-future-sales/\
input/competitive-data-science-predict-future-sales/data1.pkl')    

In [1]:
matrix.head()

NameError: name 'matrix' is not defined

In [30]:
use_cols = ['date_block_num', 'item_category_id', 'item_id', 'shop_id',
            'total_sales', 'item_cnt_day', 'item_cnt_month', 'date_mean_item_cnt2', 
            'date_mean_item_cnt3', 'date_mean_item_cnt6', 'date_mean_item_cnt12',
            'date_item_mean_item_cnt2', 'date_item_mean_item_cnt3',
            'date_item_mean_item_cnt6', 'date_item_mean_item_cnt12',
            'date_shop_mean_item_cnt2', 'date_shop_mean_item_cnt3',
            'date_shop_mean_item_cnt6', 'date_shop_mean_item_cnt12',
            'date_category_mean_item_cnt2', 'date_category_mean_item_cnt3',
            'date_category_mean_item_cnt6', 'date_category_mean_item_cnt12',
            'date_category_shop_mean_item_cnt2',
            'date_category_shop_mean_item_cnt3',
            'date_category_shop_mean_item_cnt6',
            'date_category_shop_mean_item_cnt12', 'date_std_item_cnt2',
            'date_std_item_cnt3', 'date_std_item_cnt6', 'date_std_item_cnt12',
            'date_item_std_item_cnt2', 'date_item_std_item_cnt3',
            'date_item_std_item_cnt6', 'date_item_std_item_cnt12',
            'date_shop_std_item_cnt2', 'date_shop_std_item_cnt3',
            'date_shop_std_item_cnt6', 'date_shop_std_item_cnt12',
            'date_category_std_item_cnt2', 'date_category_std_item_cnt3',
            'date_category_std_item_cnt6', 'date_category_std_item_cnt12',
            'date_category_shop_std_item_cnt2', 'date_category_shop_std_item_cnt3',
            'date_category_shop_std_item_cnt6', 'date_category_shop_std_item_cnt12',
            'date_median_item_cnt2', 'date_median_item_cnt3',
            'date_median_item_cnt6', 'date_median_item_cnt12',
            'date_item_median_item_cnt2', 'date_item_median_item_cnt3',
            'date_item_median_item_cnt6', 'date_item_median_item_cnt12',
            'date_shop_median_item_cnt2', 'date_shop_median_item_cnt3',
            'date_shop_median_item_cnt6', 'date_shop_median_item_cnt12',
            'date_category_median_item_cnt2', 'date_category_median_item_cnt3',
            'date_category_median_item_cnt6', 'date_category_median_item_cnt12',
            'date_category_shop_median_item_cnt2',
            'date_category_shop_median_item_cnt3',
            'date_category_shop_median_item_cnt6',
            'date_category_shop_median_item_cnt12', 'date', 'dayofweek',
            'dayofmonth', 'month', 'quarter', 'year', '']

In [1]:
X_tra = matrix[matrix['date_block_num'] < 33][use_cols]
Y_tra = matrix[matrix['date_block_num'] < 33]['item_cnt_day']

X_val = matrix[matrix['date_block_num'] == 33][use_cols]
Y_val = matrix[matrix['date_block_num'] == 33]['item_cnt_day']

X_test = matrix[matrix['date_block_num'] == 34]['item_cnt_day']

NameError: name 'matrix' is not defined

In [33]:
params = {"max_depth": 7,
         "min_child_weight": 15,
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 4,
         "random_state": 4590}

In [34]:
tra_data = lightgbm.Dataset(X_tra, label=Y_tra)
val_data = lightgbm.Dataset(X_val, label=Y_val)

model = lightgbm.train(params, tra_data, num_boost_round=1000, \
                           valid_sets=[tra_data, val_data],  verbose_eval=100, \
                           early_stopping_rounds = 100)
pred = model.predict(X_val, num_iteration=model.best_iteration)
np.sqrt(mean_squared_error(pred, Y_val))

Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 3.57956	valid_1's rmse: 9.67702
[200]	training's rmse: 1.79539	valid_1's rmse: 8.11559
[300]	training's rmse: 1.31447	valid_1's rmse: 7.7047
[400]	training's rmse: 1.20664	valid_1's rmse: 7.47135
[500]	training's rmse: 1.1813	valid_1's rmse: 7.35806
[600]	training's rmse: 1.15736	valid_1's rmse: 7.26794
[700]	training's rmse: 1.13769	valid_1's rmse: 7.2182
[800]	training's rmse: 1.12335	valid_1's rmse: 7.1926
[900]	training's rmse: 1.10732	valid_1's rmse: 7.19075
[1000]	training's rmse: 1.09478	valid_1's rmse: 7.1737
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 1.09478	valid_1's rmse: 7.1737


7.173696085414452